In [52]:
#SETUP always run this
import math
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
path = "stockcharts.csv"

allData = pd.read_csv(path)

#drop rows where sector is Nan
allData = allData.dropna(subset=['Sector'])

# Create a dictionary of all the sectors mapped to a list of all the industries in that sector mapped to a list of all the tickers in that industry
sectors = {}
for index, row in allData.iterrows():
    if row['Sector'] not in sectors:
        sectors[row['Sector']] = {}
    if row['Industry'] not in sectors[row['Sector']]:
        sectors[row['Sector']][row['Industry']] = []
    sectors[row['Sector']][row['Industry']].append(row['Ticker'])


allSectors = list(sectors.keys())
allIndustries = []
for sector in allSectors:
    allIndustries.extend(list(sectors[sector].keys()))
allIndustries = list(set(allIndustries))
allTickers = []
for sector in allSectors:
    for industry in sectors[sector]:
        allTickers.extend(sectors[sector][industry])
allTickers = list(set(allTickers))



In [53]:
#display the tickers in each industry
for sector in allSectors:
    print(sector)
    for industry in sectors[sector]:
        print(industry)
        print(sectors[sector][industry])
    print()



Consumer Discretionary
Durable Household Products
['AAN', 'ACCO', 'BWMX', 'COOK', 'NPK', 'IRBT', 'CENT', 'CENTA', 'HELE', 'NWL', 'REZI', 'MIDD', 'WHR']
Auto Parts
['AEVA', 'PRTS', 'SRI', 'AXL', 'MOD', 'SMP', 'XPEL', 'ADNT', 'DAN', 'DORM', 'DRVN', 'GNTX', 'QS', 'THRM', 'VC', 'ALV', 'APTV', 'BWA', 'GPC', 'LEA', 'LKQ', 'MGA']
Business Training Agencies
['BBSI', 'HQI', 'HSII', 'KELYA', 'TBI', 'AFYA', 'CCRN', 'KFRC', 'AMN', 'ASGN', 'KFY', 'MAN', 'NSP', 'TNET']
Specialty Retailers
['BBW', 'CRMT', 'FLWS', 'HZO', 'MYTE', 'NEGG', 'RERE', 'SPWH', 'WISH', 'YSG', 'ARKO', 'CVNA', 'HIBB', 'OSTK', 'SBH', 'SPH', 'TITN', 'WINA', 'WRBY', 'ABG', 'AN', 'ASO', 'CHPT', 'EYE', 'GLBE', 'GME', 'GPI', 'IAA', 'LAD', 'MUSA', 'ODP', 'OLPX', 'RUSHA', 'RUSHB', 'RVLV', 'SAH', 'WOOF', 'AAP', 'AZO', 'BBWI', 'BBY', 'BJ', 'CHWY', 'DG', 'DKS', 'DLTR', 'FIVE', 'KMX', 'ORLY', 'PAG', 'TSCO', 'ULTA', 'WSM']
Restaurants & Bars
['BH', 'BJRI', 'CHUY', 'DENN', 'KRUS', 'LOCO', 'NATH', 'RUTH', 'ARCO', 'BROS', 'CAKE', 'DIN', 'EAT', 

In [28]:
SectorToIndustry = {}
for i in range(len(allData)):
    sector = allData.iloc[i]["Sector"]
    industry = allData.iloc[i]["Industry"]
    if sector not in SectorToIndustry:
        SectorToIndustry[sector] = [industry]
    else:
        if industry not in SectorToIndustry[sector]:
            SectorToIndustry[sector].append(industry)

SectorToIndustry

{'Consumer Discretionary': ['Durable Household Products',
  'Auto Parts',
  'Business Training Agencies',
  'Specialty Retailers',
  'Restaurants & Bars',
  'Broadline Retailers',
  'Footwear',
  'Special Consumer Services',
  'Home Construction',
  'Apparel Retailers',
  'Recreational Products',
  'Hotels',
  'Furnishings',
  'Gambling',
  'Clothing & Accessories',
  'Toys',
  'Home Improvement Retailers',
  'Automobiles',
  'Recreational Services',
  'Tires',
  'Travel & Tourism'],
 'Technology': ['Software',
  'Semiconductors',
  'Electronic Equipment',
  'Telecom Equipment',
  'Electrical Components',
  'Computer Services',
  'Computer Hardware',
  'Renewable Energy Equipment'],
 'Health Care': ['Pharmaceuticals',
  'Biotechnology',
  'Health Care Providers',
  'Medical Supplies',
  'Medical Equipment'],
 nan: [nan],
 'Financial': ['Banks',
  'Property-Casualty Insurance',
  'Consumer Finance',
  'Financial Administration',
  'Asset Managers',
  'Investment Services',
  'Full Line 

In [51]:
allData.head(20)

,Ticker,Name,Exchange,Sector,Industry,SCTR,Universe,Close,Volume
0,AAN,Aarons Company Inc.,NYSE,Consumer Discretionary,Durable Household Products,78.7,sml,14.540,86668
1,ABST,Absolute Software Corp.,NASD,Technology,Software,23.6,sml,8.630,27962
2,ABUS,Arbutus Biopharma Corp.,NASD,Health Care,Pharmaceuticals,83.3,sml,2.815,1713885
3,ACCO,Acco Brands Corp.,NYSE,Consumer Discretionary,Durable Household Products,43.5,sml,5.675,265491
4,ACET,"Adicet Bio, Inc.",NASD,Health Care,Biotechnology,15.4,sml,7.890,125499
5,ACHR,Archer Aviation Inc.,NYSE,NaN,NaN,48.8,sml,2.960,1325404
6,ACMR,"ACM Research, Inc.",NASD,Technology,Semiconductors,20.4,sml,10.405,584218
7,ACNB,Acnb,NASD,Financial,Banks,76.8,sml,37.775,1204
8,ACRE,Ares Commercial Real Estate Corp.,NYSE,Real Estate,Mortgage REITs,43.1,sml,11.355,412491
9,ACRV,Acrivon Therapeutics Inc.,NASD,Health Care,Pharmaceuticals,NaN,NaN,19.740,27031
